In [1]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers

def get_mnist_model():
    inputs = keras.Input(shape=(28*28,))
    features = layers.Dense(512, activation="relu")(inputs)
    features = layers.Dropout(0.5)(features)
    outputs = layers.Dense(10, activation="softmax")(features)
    model = keras.Model(inputs, outputs)
    return model

(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28*28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28*28)).astype("float32")/255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"]
)
model.fit(train_images, train_labels,
         epochs=3,
         validation_data=(val_images, val_labels))
test_metrics = model.evaluate(test_images, test_labels)
predictions = model.predict(test_images)

Epoch 1/3
1563/1563 [==============================] - 17s 8ms/step - loss: 0.2932 - accuracy: 0.9125 - val_loss: 0.1541 - val_accuracy: 0.9559
Epoch 2/3
1563/1563 [==============================] - 13s 8ms/step - loss: 0.1630 - accuracy: 0.9552 - val_loss: 0.1161 - val_accuracy: 0.9669
Epoch 3/3
313/313 [==============================] - 1s 3ms/step


## using custom metrics

In [5]:
import tensorflow as tf

class RootMeanSquaredError(keras.metrics.Metric):
#define the state of the variables in the constructor    
    def __init__(self, name="rmse", **kwargs):
        super().__init__(name=name, **kwargs)
        self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
        self.total_samples = self.add_weight(
            name="total_samples", initializer="zeros", dtype="int32"
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        #y_pred corresponds to predictions from the model
        y_true= tf.one_hot(y_true, depth=tf.shape(y_pred)[1]) #targets/lables for one batch
        mse = tf.reduce_sum(tf.square(y_true - y_pred))
        self.mse_sum.assign_add(mse)
        num_samples = tf.shape(y_pred)[0]
        self.total_samples.assign_add(num_samples)

    def result(self):
        #returns the current value of the metric
        return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))
    
    def reset_state(self):
        self.mse_sum.assign(0.)
        self.total_samples.assign(0)

model = get_mnist_model()
model.compile(optimizer="rmsprop",
             loss="sparse_categorical_crossentropy",
             metrics=["accuracy", RootMeanSquaredError()])
model.fit(train_images, train_labels,
         epochs=3,
         validation_data=(val_images, val_labels))
test_metrics = model.evaluate(test_images, test_labels)

Epoch 1/3
1563/1563 [==============================] - 19s 12ms/step - loss: 0.2972 - accuracy: 0.9120 - rmse: 7.1830 - val_loss: 0.1488 - val_accuracy: 0.9556 - val_rmse: 7.3512
Epoch 2/3
1563/1563 [==============================] - 21s 14ms/step - loss: 0.1671 - accuracy: 0.9522 - rmse: 7.3562 - val_loss: 0.1253 - val_accuracy: 0.9675 - val_rmse: 7.4042
Epoch 3/3
313/313 [==============================] - 1s 2ms/step - loss: 0.1087 - accuracy: 0.9721 - rmse: 7.4327


## using the callback arguments in fit() method

In [6]:
#callbacks are paased to the model via the callback argument in fit()
#which takes a list of callbacks. You can pass any number of arguments
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=2,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="checkpoint_path.keras",
        monitor="val_loss",
        save_best_only=True,
    )
]

model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"]
)
model.fit(train_images, train_labels,
         epochs=10,
          callbacks=callbacks_list,
         validation_data=(val_images, val_labels))
test_metrics = model.evaluate(test_images, test_labels)
predictions = model.predict(test_images)

Epoch 1/10
1563/1563 [==============================] - 6s 4ms/step - loss: 0.2982 - accuracy: 0.9114 - val_loss: 0.1621 - val_accuracy: 0.9496
Epoch 2/10
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1654 - accuracy: 0.9544 - val_loss: 0.1270 - val_accuracy: 0.9651
Epoch 3/10
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1377 - accuracy: 0.9629 - val_loss: 0.1214 - val_accuracy: 0.9694
Epoch 4/10
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1238 - accuracy: 0.9680 - val_loss: 0.1026 - val_accuracy: 0.9762
Epoch 5/10
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1150 - accuracy: 0.9714 - val_loss: 0.1067 - val_accuracy: 0.9760
Epoch 6/10
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1079 - accuracy: 0.9736 - val_loss: 0.1123 - val_accuracy: 0.9763
Epoch 7/10
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1049 - accuracy: 0.9752 - val_loss: 0.1154 - val_accuracy: